# Preprocessing

## Featurizing

In [7]:
import mdtraj as md
import numpy as np
import datetime
import tensorflow as tf
import nglview as nv

%cd /home/jovyan/ASMSA/mydev

import os, sys

repo_dir = os.getcwd()   
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

from src.utils import plot_latent_space
from src.asmsa_callbacks import callbacks
from src.vae import asmsa_beta_vae
from src.asmsa_features import process_trajectory
from src.asmsa_split import  split_dataset

nn_model = 'vae'
latent_dim = 2

[Errno 2] No such file or directory: '/home/jovyan/ASMSA/mydev'
/home/tedeschg/prj/ASMSA/mydev


In [2]:
tr = "trpcage_ds_nH.xtc"
conf = "trpcage_npt400_nH.pdb"

traj = md.load_xtc(tr, top=conf)
backbone_atoms = traj.topology.select('backbone')
traj.superpose(traj, 0, atom_indices=backbone_atoms)

<mdtraj.Trajectory with 50001 frames, 144 atoms, 20 residues, and unitcells at 0x7edb987c5b70>

In [3]:
view = nv.show_mdtraj(traj)

view.add_representation('line', selection='protein')
view

NGLWidget(max_frame=50000)

In [5]:
feat = process_trajectory(tr, conf)

## NN preprocessing

In [10]:
# Uso:
ds_train, ds_val, ds_test, ds_all, info = split_dataset(feat['features_normalized'], train_size=70, val_size=15, batch_size=64, seed=42)

Dataset Split Statistics:
  Total samples: 50001
  Train: 35000 samples (70.0%), 546 batches
  Val:   7500 samples (15.0%), 118 batches
  Test:  7501 samples (15.0%), 118 batches
  Batch size: 64
  Features shape: (50001, 558)
  Seed: 42


# VAE

In [ ]:
'''
Batch Norm, nel caso, va prima della layer activation)
'''

In [ ]:
log_dir = "logs/autoencoder/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

cb = callbacks(log_dir, latent_dim, monitor="val_loss", model=nn_model)

beta_vae, encoder, decoder = asmsa_beta_vae(
    n_features=feat['features_normalized'].shape[1], 
    latent_dim=latent_dim,
    beta=1e-7,
    )

In [12]:
beta_vae.fit(ds_train, epochs=500, validation_data=ds_val, callbacks=cb)

Warmup: start from beta = 0.000100 until beta = 1.000000 in 50 epoche.
→ Epoca 1: beta = 0.000100
Epoch 1/500


I0000 00:00:1755090745.406129  436244 service.cc:146] XLA service 0x7ed9cc01f820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755090745.406150  436244 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-08-13 15:12:25.486054: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-13 15:12:26.050490: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 45/546 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - kl_loss: 4.6555e-05 - loss: 0.2129 - reconstruction_loss: 0.2128

I0000 00:00:1755090748.673175  436244 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


532/546 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 0.0012 - loss: 0.1774 - reconstruction_loss: 0.1762
Epoch 1: val_loss improved from inf to 0.13680, saving model to vae_2d.keras
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - kl_loss: 0.0012 - loss: 0.1768 - reconstruction_loss: 0.1756 - val_kl_loss: 0.0015 - val_loss: 0.1368 - val_reconstruction_loss: 0.1353 - learning_rate: 1.0000e-04
→ Epoca 2: beta = 0.020098
Epoch 2/500
 47/546 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - kl_loss: 0.0015 - loss: 0.1366 - reconstruction_loss: 0.1351

/home/tedeschg/miniforge3/envs/asmsa/lib/python3.10/site-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


536/546 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 0.0014 - loss: 0.1353 - reconstruction_loss: 0.1339
Epoch 2: val_loss improved from 0.13680 to 0.13254, saving model to vae_2d.keras
546/546 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - kl_loss: 0.0014 - loss: 0.1353 - reconstruction_loss: 0.1339 - val_kl_loss: 0.0012 - val_loss: 0.1325 - val_reconstruction_loss: 0.1314 - learning_rate: 1.0000e-04
→ Epoca 3: beta = 0.040096
Epoch 3/500
533/546 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 0.0011 - loss: 0.1321 - reconstruction_loss: 0.1310
Epoch 3: val_loss improved from 0.13254 to 0.13138, saving model to vae_2d.keras
546/546 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - kl_loss: 0.0011 - loss: 0.1321 - reconstruction_loss: 0.1310 - val_kl_loss: 0.0010 - val_loss: 0.1314 - val_reconstruction_loss: 0.1304 - learning_rate: 1.0000e-04
→ Epoca 4: beta = 0.060094
Epoch 4/500
535/546 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 9.8410e-04 - loss: 0.1311 - reconstruction_loss: 0.1302
Epoch 4: val_loss improved fro

In [13]:
src = f"vae_{latent_dim}d.keras"
dest = "/home/tedeschg/prj/ASMSA/mydev/models/"

!mv {src} {dest}

tensorboard --logdir logs/autoencoder --host localhost --port 6006

# Decode and visualize

In [ ]:
encoder.save('./models/encoder_vae.keras')
decoder.save('./models/decoder_vae.keras')


In [ ]:
for batch_x, _ in ds_test.take(1):
    sample_x = batch_x[30]  
    break
    
sample_x_batch = tf.expand_dims(sample_x, axis=0)
z_mean, z_log_var, test = encoder.predict(sample_x_batch)

z_mean

In [ ]:
target = np.array([0,-2]).reshape(1, latent_dim)
emb, sample = plot_latent_space(latent_dim, encoder, ds_all, conf, tr, test, bb_indices, model=nn_model, exact=False)

In [ ]:
rms_ref = md.load_pdb(conf)
rms_ref_bb  = rms_ref.atom_slice(bb_indices)
rms_ref_ca  = rms_ref.atom_slice(ca_indices)
rms_tr = md.load_xtc(tr, top=rms_ref)
rmsd = md.rmsd(rms_tr, rms_ref)

In [ ]:
p_indices = traj.topology.select("protein")
n_p = len(p_indices)

In [ ]:
s = decoder.predict(sample)

coords_size = coords.shape[1]  # dimensione delle coordinate
angles_size = s.shape[1] - coords_size  # dimensione degli angoli

# Separa coordinate e angoli
s_coords = s[:, :coords_size]
s_angles = s[:, coords_size:]

# Inverti le trasformazioni separatamente
coords_orig = scaler_coords.inverse_transform(s_coords)
angles_orig = scaler_angles.inverse_transform(s_angles)

# Prendi solo le coordinate per la ricostruzione
coords_flat = coords_orig[0, :]                    
coords_p = coords_flat.reshape((n_p, 3))
mask_bb = np.isin(p_indices, bb_indices)
coords_bb = coords_p[mask_bb] 
coords_ca = coords_bb[1::4] 

new_traj = md.Trajectory(
    xyz=np.array([coords_bb]),     
    topology=rms_ref_bb.topology     
)

new_traj.save_pdb("./models/vae_reconstructed.pdb")

In [ ]:
view = nv.show_file('./models/vae_reconstructed.pdb')
view.clear_representations()
view.add_line() 
#view.add_cartoon()
view.center()
view

In [ ]:
mse = np.mean((s[0] - sample_x.numpy())**2)
print("MSE ricostruzione:", mse)

In [ ]:
from src.asmsanalysis import analyze_reconstruction, plot_section_errors
import matplotlib.pyplot as plt
# Utilizzo con i tuoi dati
# Assumendo che sample_x e s[0] siano definiti
orig = sample_x.numpy() 
recon = s[0]

# Analisi completa
fig, metrics = analyze_reconstruction(orig, recon, title_prefix="Beta-VAE ")

# Solo errori per sezione
fig_sections, section_stats = plot_section_errors(orig, recon, n_sections=25, 
                                                   title="Analisi Errori per Sezione")

plt.show()

# Stampa metriche
print("\n=== METRICHE RICOSTRUZIONE ===")
for metric, value in metrics.items():
    (f"{metric.upper()}: {value:.6f}")

In [ ]:
import nglview as nv

# Crea un widget vuoto
view = nv.NGLWidget()

# --- primo modello -------------------------------------------------
comp1 = view.add_component("./models/vae_reconstructed.pdb")   # oppure percorso al tuo 1° PDB
comp1.clear_representations()                         # opzionale
comp1.add_representation("line", color="skyblue")  # o 'line', 'surface', ecc.

# --- secondo modello ----------------------------------------------
comp2 = view.add_component("./models/ae_reconstructed.pdb")       # secondo PDB da sovrapporre
comp2.clear_representations()
comp2.add_representation("line", color="orange")   # scegli un colore diverso

# Centra e mostra
view.center()
view
